In [ ]:
import geopandas as gpd
import pandas as pd
from crate import client
from shapely.geometry import shape
import datetime
import os

In [99]:
# Create connection
connectionHeal = client.connect(servers="http://localhost:4200", username="crate", error_trace=True)

# open DB cursor
cursorHeal = connectionHeal.cursor()

query = "SELECT entity_id, dateobserved, stationname, description, location, temperature, relativehumidity, dewpoint, atmosphericpressure, solarradiation, precipitation, windspeed, winddirection FROM doc.etweatherobserved WHERE dateobserved >= 1701388800000 AND dateobserved <= 1704067199999"

# execute request
cursorHeal.execute(sql=query)

# create geodataframe
gdf_conn = gpd.GeoDataFrame(data=cursorHeal.fetchall(), columns=pd.DataFrame(data=cursorHeal.description)[0],)

In [ ]:
# Convert 'dateobserved' from Unix timestamp to human-readable datetime
gdf_conn["dateobserved"]

gdf_conn['dateobserved'] = pd.to_datetime(
    gdf_conn['dateobserved'], 
    unit='ms',
    origin='unix' 
)

In [ ]:
# check if entire month was included
gdf_conn.sort_values(by='dateobserved')

,entity_id,dateobserved,stationname,description,location,temperature,relativehumidity,dewpoint,atmosphericpressure,solarradiation,precipitation,windspeed,winddirection
82049,hd:DE_Heidelberg_69117_1:WeatherObserved,2023-12-01 00:00:00.000,69117_Königstuhl_10,Landessternwarte - Königstuhl,"{'coordinates': [8.724495, 49.398345], 'type':...",-0.395,90.60,NaN,NaN,NaN,0.10,1.020000,104.0
217635,hd:DE_Heidelberg_69117_5:WeatherObserved,2023-12-01 00:00:00.000,69117_Rodelweg,Königstuhl Winterdienst,"{'coordinates': [8.728104, 49.403737], 'type':...",-0.390,NaN,-0.93,NaN,NaN,0.10,0.583333,112.3
82046,hd:DE_Heidelberg_69115_4:WeatherObserved,2023-12-01 00:00:00.000,69115_Czernyring_22/11-12,Institutsdach Czernyring,"{'coordinates': [8.6761, 49.400672], 'type': '...",0.800,93.00,NaN,983.90,NaN,0.10,1.400000,198.0
13077,hd:DE_Gaiberg_69251_21:WeatherObserved,2023-12-01 00:00:00.000,69251_In_den_Weinäckern,Gaiberg-Berghof,"{'coordinates': [8.74322, 49.36094], 'type': '...",0.307,92.40,NaN,960.00,NaN,0.10,0.850000,67.0
13078,hd:DE_Heidelberg_69115_2:WeatherObserved,2023-12-01 00:00:00.000,69115_Poststraße_15,Stadtbücherei,"{'coordinates': [8.68671, 49.40666], 'type': '...",1.300,92.80,NaN,NaN,NaN,0.10,1.500000,205.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
288629,hd:DE_Heidelberg_46:WeatherObserved,2023-12-31 23:58:58.352,69126_Boxbergring_16,MeteoHelixR_21_Boxberg_DAH,"{'coordinates': [8.703570385, 49.374008928], '...",6.300,83.80,NaN,980.20,0.0,0.00,NaN,NaN
143536,hd:DE_Heidelberg_69118_6:WeatherObserved,2023-12-31 23:59:00.000,69118_Wendestelle_Hbw,Peterstal Winterdienst,"{'coordinates': [8.751095, 49.452689], 'type':...",5.690,80.00,2.50,NaN,NaN,0.00,2.025000,107.0
67414,hd:DE_Heidelberg_69118_23:WeatherObserved,2023-12-31 23:59:00.000,69118_B37,Schlierbach Winterdienst,"{'coordinates': [8.777746, 49.4036537], 'type'...",7.080,78.80,3.71,NaN,NaN,0.02,2.100000,109.2
266969,hd:DE_Heidelberg_69117_5:WeatherObserved,2023-12-31 23:59:00.000,69117_Rodelweg,Königstuhl Winterdienst,"{'coordinates': [8.728104, 49.403737], 'type':...",3.600,96.01,3.02,NaN,NaN,0.00,2.830555,218.9


In [ ]:
# Convert 'location' GeoJSON dicts to Shapely geometries
gdf_conn['geometry'] = gdf_conn['location'].apply(shape)

# Set the new 'geometry' column as the active geometry
gdf_conn = gdf_conn.set_geometry('geometry')

# Set CRS
gdf_conn.set_crs(epsg=4326, inplace=True)

# Now save
gdf_conn.to_file("../data/temp_HD/sensor_data_20230112_31days.gpkg", 
                     layer='dec_temp', 
                     driver="GPKG")

# Month that are already downloaded

Year 2023:

Downloaded:
- Jan
- Feb
- Mar
- Apr
- May
- Jun
- July
- Aug
- Sep
- Oct
- Nov
- Dec

Current:

Open to download:
